#PRKTIKUM 8
# Evaluasi Beberapa Algoritma Klasifikasi

Tujuan :
Setelah mengikuti praktikum ini praktikan
1.   Memahasi Evaluasi beberapa metrik untuk mengukur kinerja klasifikasi
2.   Mahasiswa dapat memahami confisin matrik dan perananya untuk evaluasi klasifikasi
3. Memehami dan mampu menggunakan evaluasi k-fold








##Dasar Teori

Validasi klasifikasi

Metode klasifikasi dibangun untuk dapat belajar dan diharapakan dapat membuat prediksi atau keputusan suatu data baru berdasarkan data yang sudah ada. Proses pembelajaran terhadap dataset yang sudah ada untuk mendapatkan model. Selanjutnya suatu data baru yang belum teridentifikasi dapat ditentukan label atau kelasnya berdasarkan model yang telah dibangun.
Proses pembelajaran yang dibangun terhadap suatu datasaet dilakukan dengan cara membagi suatu data menjadi dua bagian yakni sebagian digunakan sebagai data pelatihan dan sebagian lainnya sebagai data validasi. Besaran jumlah masing-masing data pelatihan dan data validasi dapat ditentukan (50%,50%) atau (75%,25%).
Tahap selanjutnya data pelatihan akan digunakan untuk membagun model dengan menentukan parameter terbaik dari masing-masing model berdasrakan dataset yang ada. Kinerja terbaik dari suatu model yang dikembangkan diukur menggunakan beberap tolok ukur sebagai berikut :
- presison
- recall
- specifity
- akurasi
Masing-masing metrik tersebut ditentukan berdasarkan matrik kebingungan.



Confusion matrik
Mtriks kebingungan atau matriks kesalahan adalah suatu tabel yang merepresentasikan kinerja suatu algoritma. Setiap baris matriks mewakili instance di kelas yang diprediksi sementara setiap kolom mewakili instance di kelas yang sebenarnya (atau sebaliknya).

#Beberapa tahapan dalam validasi klasifikasi


Beberapa metode klasifikasi yang telah dipelajari mempunyai kelebihan dan kekurannya. Perbandingan kinerja antar metode klasifikasi ini diukur menggunakan beberapa metrik.  Pada praktikum ini akan dibahas beberapa metrik untuk mengukur kinerja klasifikasi.
Tahapan untuk mengevaluasi suatu metode klasifikasi dilakukan sebagai berikut :
- Pisahkan set data validasi.
- Siapkan test harness untuk menggunakan validasi silang 10 kali lipat.
- Buat beberapa model berbeda untuk memprediksi spesies dari pengukuran bunga
- Pilih model terbaik.

##1. Tentukan metode klasifikasi yang akan dievaluasi

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


In [2]:
# Spot Check Algorithms
models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []

##2. Baca dataset
Pada contoh ini terdapat tiga buah dataset yakni :
- dataR2
- iris data
- breast cancer

Kita akan gunakan dataR2 sebagai contoh sedangkan dua data berikunya akan digunakan sabagai latihan.

In [3]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv
# ! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
#! wget https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv


--2024-05-09 12:03:14--  https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘dataR2.csv’

dataR2.csv              [ <=>                ]   7.49K  --.-KB/s    in 0s      

2024-05-09 12:03:14 (92.8 MB/s) - ‘dataR2.csv’ saved [7665]



In [4]:
import pandas as pd
import numpy as np
# dataset = pd.read_csv('iris.csv')
# dataset = pd.read_csv('breast_cancer.csv')
dataset = pd.read_csv('dataR2.csv')
dataset

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1
...,...,...,...,...,...,...,...,...,...,...
111,45,26.850000,92,3.330,0.755688,54.6800,12.100000,10.96000,268.230,2
112,62,26.840000,100,4.530,1.117400,12.4500,21.420000,7.32000,330.160,2
113,65,32.050000,97,5.730,1.370998,61.4800,22.540000,10.33000,314.050,2
114,72,25.590000,82,2.820,0.570392,24.9600,33.750000,3.27000,392.460,2


In [5]:
dataset.head()

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1


##3.1 Membuat Set Data Validasi
Tujuan evaluasi adalah memperkirakan keakuratan model terbaik pada data tak terlihat dengan mengevaluasinya pada data tak terlihat aktual. Evaluasi ini dilakukan dengan cara membagi dataset menjadi dua bagian yakni sebagian digunakan sebagai data latih dan sebagian yang lain digunakan sebagai data uji. Beberapa praktikum sebelumnya telah dipraktekkan dengan menggunakan metode train_test_split :
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)


Sebelum membuat data latih dan data uji. Langkah pertama pisahkan antara data attribute dan data label suatu dataset. Karena datasaet data2R terdiri dari 9 attribut dan 1 label.  Maka kita catat X sebagai data dan Y sebagai label.

In [6]:
from sklearn.model_selection import train_test_split
array = dataset.values
X = array[:,0:9]
y = array[:,9]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)


##3.2 K-Fold Cross Validation
k-Fold Cross-Validation Validasi silang adalah prosedur pengambilan sampel ulang yang digunakan untuk mengevaluasi model pembelajaran mesin pada sampel data terbatas. Prosedur ini memiliki parameter tunggal yang disebut k yang mengacu pada jumlah grup yang akan dipecah menjadi sampel data tertentu. Karena itu, prosedur ini sering disebut k-fold cross-validation. Ketika nilai spesifik untuk k dipilih, nilai tersebut dapat digunakan sebagai pengganti k dalam referensi ke model, seperti k = 10 menjadi validasi silang 10 kali lipat.

Validasi silang terutama digunakan dalam pembelajaran mesin terapan untuk memperkirakan kemampuan model pembelajaran mesin pada data yang tidak terlihat. Yaitu, menggunakan sampel terbatas untuk memperkirakan bagaimana performa model secara umum saat digunakan untuk membuat prediksi pada data yang tidak digunakan selama pelatihan model.

Prosedur umumnya adalah sebagai berikut:
- Acak kumpulan data berdasarkan id
- Pisahkan kumpulan data menjadi kelompok k
- Untuk setiap grup unik:
  - Ambil kelompok sebagai kelompok data uji
  - Ambil kelompok yang tersisa sebagai kumpulan data pelatihan
  - Catat hasil pengujian masing-masing group

K-Fold ini digunakan untuk validasi model yang digunakan. Evaluasi dilakukan pada dataset latih. Pada praktikum ini digunakan k=10, berarti akan membagi dataset latih menjadi 10 bagian. Satu bagian digunakan untuk testing dan sisanya sebagai data latih.

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

# List model
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))


KNN: 0.487778 (0.143540)
NB: 0.662222 (0.125629)
SVM: 0.465556 (0.047778)


##4. Buat Prediksi

Kita harus memilih algoritma yang akan digunakan untuk membuat prediksi.

##4.1 Membuat Prediksi
Kita dapat menyesuaikan model pada seluruh dataset pelatihan dan membuat prediksi pada dataset validasi.

In [8]:
model = SVC(gamma='auto')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

# 4.2 Evaluasi Prediksi
Kami dapat mengevaluasi prediksi dengan membandingkannya dengan hasil yang diharapkan dalam set validasi, lalu menghitung akurasi klasifikasi, serta matriks kebingungan dan laporan klasifikasi.

In [9]:
# create dataset
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

print(X)
print(y)


[[  2.47475454   0.40165523   1.68081787 ...  -6.59044146  -2.21290585
   -3.139579  ]
 [  0.84802507   2.81841945  -2.76008732 ...   3.00844461   0.78661954
   -1.27681551]
 [ -1.90041246  -0.56901823  -1.76220236 ...   3.37336417  -2.28613707
    1.90344983]
 ...
 [  0.7673844   -2.91920559   2.80851577 ...   4.42591832   0.46321196
   -3.30523346]
 [  2.05510667  -0.99009741   0.73577291 ...   3.05100898  -1.40715279
   -0.51579331]
 [-10.96847792  -2.39810735  -0.96700953 ... -11.16298557   1.16646392
    0.60835176]]
[0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 1 0 1 0 0 1
 1 0 1 0 0 1 1 1 0 1 0 0 1 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1
 0 0 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1 1 0 0
 0 0 1 1 1 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1
 0 1 1 0 1 1 0 1 0 1 0

In [10]:
from sklearn.metrics import classification_report
print(classification_report(Y_validation, predictions))

              precision    recall  f1-score   support

         1.0       0.25      1.00      0.40         6
         2.0       0.00      0.00      0.00        18

    accuracy                           0.25        24
   macro avg       0.12      0.50      0.20        24
weighted avg       0.06      0.25      0.10        24



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Tugas :

1. Lakukan perbandingan klasifikasi antara SVM, Naive Bayes, KNN untuk dataset iris dan breast cancer
2. Pada kedua dataset lakukan proses normaslisai data dan preproses data untuk menengani data yang hilang jika ada.

A. Data Iris

In [11]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-05-09 12:04:48--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-05-09 12:04:48 (42.4 MB/s) - ‘iris.csv’ saved [3975/3975]



In [12]:
iris = pd.read_csv('iris.csv')
iris

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [13]:
iarray = iris.values
Xi = iarray[:,0:4]
yi = iarray[:,4]
iX_train, iX_validation, iY_train, iY_validation = train_test_split(Xi, yi, test_size=0.20, random_state=1, shuffle=True)

In [14]:
for namei, modeli in models:
  kfoldi = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
  cvi_results = cross_val_score(modeli, iX_train, iY_train, cv=kfoldi, scoring='accuracy', error_score='raise')
  results.append(cvi_results)
  names.append(namei)
  print('%s: %f (%f)' % (namei, cvi_results.mean(), cvi_results.std()))

KNN: 0.958333 (0.045644)
NB: 0.950000 (0.031180)
SVM: 0.983333 (0.020412)


In [15]:
modeli = SVC(gamma='auto')
modeli.fit(iX_train, iY_train)
predictionsi = modeli.predict(iX_validation)

In [16]:
Xi, yi = make_classification(n_samples=1000, n_features=20, n_informative =15, n_redundant=5, random_state=1)
print(classification_report(iY_validation, predictionsi))

              precision    recall  f1-score   support

      Setosa       1.00      1.00      1.00        11
  Versicolor       1.00      0.92      0.96        13
   Virginica       0.86      1.00      0.92         6

    accuracy                           0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30



B. Data Breast Cancer

In [17]:
! wget https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv

--2024-05-09 12:08:01--  https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3664 (3.6K) [text/plain]
Saving to: ‘breast_cancer.csv’

breast_cancer.csv   100%[===================>]   3.58K  --.-KB/s    in 0s      

2024-05-09 12:08:01 (29.0 MB/s) - ‘breast_cancer.csv’ saved [3664/3664]



In [18]:
bc = pd.read_csv('breast_cancer.csv')
bc

,id,clump_thickness,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nucklei,bland_chromatin,normal_nucleoli,mitoses,diagnosis
0,1000025,5.0,1.0,NaN,1.0,NaN,1.0,3.0,1.0,1.0,benign
1,1002945,5.0,4.0,4.0,5.0,7.0,NaN,3.0,NaN,1.0,benign
2,1015425,3.0,1.0,1.0,1.0,NaN,2.0,3.0,1.0,1.0,benign
3,1016277,6.0,8.0,8.0,1.0,3.0,NaN,3.0,7.0,1.0,benign
4,1017023,4.0,1.0,1.0,3.0,NaN,1.0,3.0,1.0,NaN,benign
...,...,...,...,...,...,...,...,...,...,...,...
95,1167439,2.0,3.0,4.0,4.0,2.0,5.0,2.0,5.0,1.0,malignant
96,1168359,8.0,2.0,3.0,NaN,6.0,3.0,7.0,1.0,1.0,malignant
97,1168736,10.0,10.0,10.0,10.0,10.0,1.0,8.0,8.0,8.0,malignant
98,1169049,7.0,3.0,4.0,4.0,3.0,3.0,3.0,2.0,7.0,malignant


In [19]:
barray = bc.values
Xb = barray[:,0:10]
yb = barray[:,10]
bX_train, bX_validation, bY_train, bY_validation = train_test_split(Xb, yb, test_size=0.20, random_state=1, shuffle=True)

In [20]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
imputed_X_train = imputer.fit_transform(bX_train)

for nameb, modelb in models:
  kfoldb = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
  cvb_results = cross_val_score(modelb, imputed_X_train, bY_train, cv=kfoldb, scoring='accuracy', error_score='raise')
  results.append(cvb_results)
  names.append(nameb)
  print('%s: %f (%f)' % (nameb, cvb_results.mean(), cvb_results.std()))

KNN: 0.587500 (0.148429)
NB: 0.900000 (0.093541)
SVM: 0.487500 (0.037500)


In [21]:
imputed_X_validation = imputer.transform(bX_validation)

modelb = SVC(gamma='auto')
modelb.fit(imputed_X_train, bY_train)
predictionsb = modelb.predict(imputed_X_validation)

In [22]:
Xb, yb = make_classification(n_samples=1000, n_features=20, n_informative =15, n_redundant=5, random_state=1)
print(classification_report(bY_validation, predictionsb))

              precision    recall  f1-score   support

      benign       0.45      1.00      0.62         9
   malignant       0.00      0.00      0.00        11

    accuracy                           0.45        20
   macro avg       0.23      0.50      0.31        20
weighted avg       0.20      0.45      0.28        20



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
